This is the code to created augmented dataset using back translation

In [ ]:
!pip install numpy requests nlpaug

     |████████████████████████████████| 405 kB 5.2 MB/s 


In [ ]:
import csv
import torch
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action
import nltk
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/hate_speech

/content/drive/MyDrive/hate_speech


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
!pip install transformers

In [ ]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)

In [ ]:
df = pd.read_csv('train_hatespeech.csv')
df.set_index('id', inplace = True)
print(df.Label.value_counts())

none      8747
sexism    2534
racism    1548
Name: Label, dtype: int64


In [ ]:
df

,Tweets,Label
0,The jokes and puns are flying free in this cam...,none
1,#MKR Lets see who the producers think are goin...,none
2,Praying Jac and Shaz do well! They're my faves...,none
3,RT @realityraver: Pete Evans the Paleo Capital...,none
4,If Kat and Andre stay tonight I will stop watc...,none
...,...,...
12829,"Yes you are. @ KenBest I swear I'm not sexist,...",sexism
12830,BACK AT YOU DOUCHE @ GrindMode407 Mannnn I'm n...,sexism
12831,Here is my review of 99% of the games I have e...,none
12832,@ Cameron _ Gray @ ILNewsFlash Western feminis...,none


In [ ]:
def augment_text(df,samples=300):
    new_text=[]
    new_label=[]

    ## data augmentation loop
    for i in tqdm(np.random.randint(0,len(df_n),samples)):
        
            text = df_n.iloc[i]['text']
            augmented_text = aug_w2v.augment(text)
            new_text.append(augmented_text)
    
    
    ## dataframe
    new=pd.DataFrame({'text':new_text,'target':1})
    df=df.append(new).reset_index(drop=True)
    return df
   
train = augment_text(train)

Back translation is very heavy since it requires transformer model inference, so for this notebook I'll only do augmentation for 200 random samples. Ideally I would definitely do it for the entire dataset but right now there's a time and resource limit. Hope this is ok for the sake of concept demo.

In [ ]:
new_text=[]
new_label=[]


for i in tqdm(np.random.randint(0,len(df), 200)):
  # print(i)
  text = df.iloc[i]['Tweets']
  label = df.iloc[i]['Label']
  augmented_text = back_translation_aug.augment(text)
  new_text.append(augmented_text)
  new_label.append(label)

## dataframe
new=pd.DataFrame({'Tweets':new_text,'Label':new_label})
df=df.append(new).reset_index(drop=True)



  0%|          | 0/200 [00:00<?, ?it/s]

11024
2815
10089
7733
10239
12378
7717
10376
11323
4876
9401
221
3280
494
2863
10455
6255
3898
5777
2592
68
6488
8937
858
9554
2153
9506
3849
10963
4009
10011
4872
12281
9938
11512
5619
2964
8011
12596
9004
10404
7858
11463
8795
5818
9706
11214
10058
11033
2824
7935
3956
7533
6462
12491
8961
5512
8812
3772
3729
4970
2423
1148
12475
7641
8025
6668
3136
5229
524
6652
11474
12383
639
718
8487
12362
11915
12671
5463
3648
7041
4083
11900
8805
5071
4526
4595
3204
5183
11380
8187
4088
11592
3283
12335
7507
8931
8281
9807
2191
11285
6655
5658
11466
6534
6031
1765
8752
5994
6498
2380
1031
12003
563
7004
5961
1959
510
3270
4208
5651
12106
7515
2718
9654
6665
5854
2595
7783
11653
2211
4782
12595
4119
10626
9996
6447
9069
631
4751
1693
9853
400
7234
51
5828
9557
9015
3254
1044
2753
3912
4562
3497
10644
1282
3667
5427
11480
7997
8359
349
7462
8303
8097
2271
4404
5
10213
2001
5882
1042
5199
4874
1344
6356
1044
114
4076
8667
4009
5951
4061
6207
4680
3076
8834
10105
9781
8829
3457
9205
6188
10063
1089

In [ ]:
df.to_csv("train_augmented.csv", index_label="id")

In [ ]:
df

,Tweets,Label
0,The jokes and puns are flying free in this cam...,none
1,#MKR Lets see who the producers think are goin...,none
2,Praying Jac and Shaz do well! They're my faves...,none
3,RT @realityraver: Pete Evans the Paleo Capital...,none
4,If Kat and Andre stay tonight I will stop watc...,none
...,...,...
13024,Suddenly 3 times as many people who could ever...,none
13025,RT @ shanselman: Reminder: Nerd Dinner / Tweet...,none
13026,@ srhbutts @ ChrisWarcraft omg omg omgMAGICAL ...,none
13027,RT @ Madasatrain69: @ OZzSue @ Muslim _ Patrol...,racism
